In [1]:
# Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive')





Mounted at /content/drive


In [13]:
import os

os.listdir("/content/brain_tumor_data/Testing")


['glioma', 'meningioma', 'notumor', 'pituitary']

In [14]:
 # Define class labels
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
class_to_label = {name: idx for idx, name in enumerate(class_names)}

print(class_to_label)


{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


In [15]:
import numpy as np
from PIL import Image
import os

def load_dataset(base_path, class_to_label, img_size=150):
    data = []
    labels = []

    for class_name in os.listdir(base_path):
        class_path = os.path.join(base_path, class_name)
        label = class_to_label[class_name]
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            try:
                img = Image.open(img_path).resize((img_size, img_size)).convert('RGB')
                data.append(np.array(img))
                labels.append(label)
            except Exception as e:
                print(f"Skipped {img_path} due to error: {e}")

    return np.array(data) / 255.0, np.array(labels)


In [16]:
# Set the paths
train_path = "/content/brain_tumor_data/Training"
test_path = "/content/brain_tumor_data/Testing"

# Load the data
X_train, y_train = load_dataset(train_path, class_to_label)
X_test, y_test = load_dataset(test_path, class_to_label)

# Confirm shapes
print("✅ Data loaded successfully")
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Testing data shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)


✅ Data loaded successfully
Training data shape: (5712, 150, 150, 3)
Training labels shape: (5712,)
Testing data shape: (1311, 150, 150, 3)
Testing labels shape: (1311,)


In [17]:
from tensorflow.keras.utils import to_categorical

# One-hot encode the labels
y_train_encoded = to_categorical(y_train, num_classes=4)
y_test_encoded = to_categorical(y_test, num_classes=4)

# Check shapes
print("One-hot encoded training labels shape:", y_train_encoded.shape)
print("One-hot encoded testing labels shape:", y_test_encoded.shape)


One-hot encoded training labels shape: (5712, 4)
One-hot encoded testing labels shape: (1311, 4)


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')  # 4 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,868 (18.42 MB)

 Trainable params: 4,828,868 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
history = model.fit(
    X_train, y_train_encoded,
    epochs=10,
    batch_size=32,
    validation_split=0.1
)


Epoch 1/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.5634 - loss: 1.0022 - val_accuracy: 0.9038 - val_loss: 0.4479
Epoch 2/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 260s 2s/step - accuracy: 0.8050 - loss: 0.4737 - val_accuracy: 0.9318 - val_loss: 0.2913
Epoch 3/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - accuracy: 0.8882 - loss: 0.3008 - val_accuracy: 0.9336 - val_loss: 0.2286
Epoch 4/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.9204 - loss: 0.2076 - val_accuracy: 0.9668 - val_loss: 0.1354
Epoch 5/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - accuracy: 0.9426 - loss: 0.1486 - val_accuracy: 0.9266 - val_loss: 0.2616
Epoch 6/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - accuracy: 0.9617 - loss: 0.1045 - val_accuracy: 0.9650 - val_loss: 0.1410
Epoch 7/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.9728 - loss: 0.0747 - val_accuracy: 0.9773 - val_loss: 0.1157
Epoch 8/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 0.9792 - loss: 0.0558 - val_accu

In [20]:
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f"✅ Test Accuracy: {test_accuracy*100:.2f}%")


41/41 ━━━━━━━━━━━━━━━━━━━━ 17s 422ms/step - accuracy: 0.9508 - loss: 0.1662
✅ Test Accuracy: 96.11%


In [23]:
model.save("brain_tumor_cnn_model.h5")
print("✅ Model saved as brain_tumor_cnn_model.h5")


✅ Model saved as brain_tumor_cnn_model.h5


In [24]:
# Save to your Drive
!cp brain_tumor_cnn_model.h5 /content/drive/MyDrive/


In [25]:
from google.colab import files
files.download("brain_tumor_cnn_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
from tensorflow.keras.models import load_model
model = load_model("brain_tumor_cnn_model.h5")
print(model.input_shape)


(None, 150, 150, 3)
